In [ ]:
pip install tensorview

In [ ]:
import tensorflow as tf
from keras.datasets import cifar10
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from skimage import io
import tensorview as tv
import tarfile
import glob
import argparse
import os, time, datetime
#import PIL.Image as Image
from keras.models import load_model, model_from_json
from skimage.metrics import peak_signal_noise_ratio,structural_similarity
from skimage.io import imread, imsave
import cv2

Here we download the testing dataset and then convert the images into suitable size using tensorflow functions.

In [ ]:
!wget http://robotics.ethz.ch/~asl-datasets/flir_17_Sept_2013/asl_eth_flir.zip

In [ ]:
!unzip asl_eth_flir.zip

In [ ]:
#Declaration of list that will contain the testing files
files = []

#Fetching the files of the images
for i in range(5,13):
  img_dir = "flir_17_Sept_2013/Sempach-{}/8bit".format(i)
  data_path = os.path.join(img_dir,'*g')
  files_ = glob.glob(data_path)
  files.extend(files_)

In [ ]:
#Displaying an image from the testing dataset
im = cv2.imread(files[0])
plt.imshow(im)
plt.show()


In [ ]:
#Printing total number of testing files
print(len(files))

In [ ]:
#Declaration of list which will hold low res testing images
test_data_lr = []
#Declaration of list which will hold high res testing images
test_data_hr = []

def random_crop(img, random_crop_size): 
    """Function used for generating random cropped images"""
    h, w = img.shape[0], img.shape[1] 
    dy, dx = random_crop_size 
    x = np.random.randint(0, w-dx + 1) 
    y = np.random.randint(0, h-dy + 1) 
    return img[y:(y+dy), x:(x+dx), :] 

 
def generate_test_images_1(test_files,test_lr,test_hr):
  """Generation of images with the help of tensorflow random contrast and resizing functions"""
  for f in test_files:
    im = cv2.imread(f)
    try:
      img_1 = tf.image.resize_with_crop_or_pad(im,256,256)
      img_2 = tf.image.random_contrast(img_1,0.2,0.21)   
      test_lr.append(np.asarray(img_2))
      test_hr.append(np.asarray(img_1))
    except:
      print('image cannot be displayed')   
  return test_lr,test_hr    

def generate_test_images_2(test_files,test_lr,test_hr):
  """Generation of 2048 grayscale images with the help of random contrast and random crop numpy functions"""
  for i in range(2048):
    try:
      im = cv2.imread(test_files[i])
      gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
      gray = gray/255.0
      gray = np.expand_dims(gray,axis = -1)  
      img_0 = np.concatenate((gray,gray,gray),axis = -1)
      img_1 = random_crop(img_0,(256,256))
      img_2 = tf.image.random_contrast(img_1,0.2,0.21)
      test_hr.append(np.asarray(img_1))
      test_lr.append(np.asarray(img_2))
    except:
      print('cannot display') 
  return test_lr,test_hr    

#test_data_lr,test_data_hr = generate_test_images_2(files,test_data_lr,test_data_hr)
test_data_lr,test_data_hr = generate_test_images_1(files,test_data_lr,test_data_hr)

In [ ]:
#Checking the size of low res testing images 
print(np.asarray(test_data_lr).shape)


# TRAIN DATASET 
Downloading the training dataset which is the coco dataset because of less availability of RAM I am using the validation dataset which has 5000 images.

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip

In [ ]:
!unzip val2017.zip

In [ ]:
import glob,os
img_dir = "val2017" 
data_path = os.path.join(img_dir,'*g')
files2 = glob.glob(data_path)

In [ ]:
#Printing the number of training files
print(len(files2))

In [ ]:
data2 = []
data = []
def crop_img(img):
  """Converting images into grayscale and adding random contrast with the help of tensorflow functions"""
  img_2 = tf.image.random_contrast(img,0.2,0.21)
  img_1 = tf.image.rgb_to_grayscale(img)
  img_2 = tf.image.rgb_to_grayscale(img_2)
  img_1 = tf.concat([img_1,img_1,img_1],axis = -1)
  img_2 = tf.concat([img_2,img_2,img_2],axis = -1)
  return img_1,img_2

def generate_images(files,list1,list2):
  """Resizing training images and cropping them  """
  for i in range(5000):
    try:
      im = cv2.imread(files[i])
      im = cv2.resize(im,(128,128),interpolation = cv2.INTER_AREA)
      im2,im3 = crop_img(im)
      target_img = im2
      input_img = im3
      list1.append(np.asarray(target_img))
      list2.append(np.asarray(input_img))    
    except:
      print('Cannot work on the image')
  return list1,list2

data,data2 = generate_images(files2,data,data2)


In [ ]:
#Converting the training images list and labels list into numpy arrays
data_array = np.asarray(data)
data2_array = np.asarray(data2)

In [ ]:
#Printing the shape of training images array
print(data_array.shape)

In [ ]:
def wasserstein_loss(y_true, y_pred):
    """Code for wasserstein loss function"""
    return -tf.math.reduce_mean(y_true - y_pred)

In [ ]:
def gen_sub(img_shape,filters = 64):
  """Generator block where enhanced infrared images are generated"""
  input_img = tf.keras.Input(shape=img_shape)
  input_image = tf.keras.layers.Conv2D(filters=64,kernel_size=(4,4),strides=2,padding = 'same')(input_img)
  c1 = tf.keras.layers.ReLU()(input_image)
  p1 = c1
  c1 = tf.keras.layers.Conv2D(filters=filters,kernel_size=(3,3),strides=1,padding = 'same')(c1)
  c1 = tf.keras.layers.BatchNormalization()(c1)
  c2 = tf.keras.layers.ReLU()(c1)
  c2 = tf.keras.layers.Conv2D(filters=filters,kernel_size=(3,3),strides=1,padding = 'same')(c2)
  c2 = tf.keras.layers.BatchNormalization()(c2)
  c3 = tf.keras.layers.ReLU()(c2)  
  u1 = tf.concat([c3,p1],axis = -1)
  u2 = tf.keras.layers.Conv2D(filters=filters,kernel_size=(1,1),strides=1)(u1)
  u3 = tf.keras.layers.Conv2DTranspose(filters = 3,kernel_size=(4,4),strides = 2,padding = 'same')(u2)
  u4 = tf.keras.activations.tanh(u3)
  gen_sub = tf.keras.Model(input_img,u4)
  return gen_sub

In [ ]:
def disc_sub(img_dim,filters = 64):
  """Discriminator block where generated images are discriminated against the given images"""
  input_src_img = tf.keras.Input(shape = img_dim)
  input_target_img = tf.keras.Input(shape = img_dim)
  merged = tf.keras.layers.Concatenate()([input_src_img,input_target_img])
  c1 = tf.keras.layers.Conv2D(filters = filters,kernel_size=(4,4),strides=2,padding='same')(merged)
  c2 = tf.keras.layers.LeakyReLU()(c1)
  c2 = tf.keras.layers.Conv2D(filters = filters*2,kernel_size=(4,4),strides=2,padding='same')(c2)
  c2 = tf.keras.layers.BatchNormalization()(c2)
  c3 = tf.keras.layers.LeakyReLU()(c2)
  c3 = tf.keras.layers.Conv2D(filters = filters*4,kernel_size=(4,4),strides=2,padding='same')(c3)
  c3 = tf.keras.layers.BatchNormalization()(c3)
  c4 = tf.keras.layers.LeakyReLU()(c3)
  c4 = tf.keras.layers.Conv2D(filters = filters*8,kernel_size=(4,4),strides=1,padding='same')(c4)
  c4 = tf.keras.layers.BatchNormalization()(c4)
  c5 = tf.keras.layers.LeakyReLU()(c4) 
  c6 = tf.keras.layers.Conv2D(filters = 1,kernel_size=(4,4),strides=1,padding='same')(c5)
  c7 = tf.keras.activations.sigmoid(c6)
  disc = tf.keras.Model([input_src_img,input_target_img],c7)
  optim = tf.keras.optimizers.Adagrad(learning_rate = 0.002)
  disc.compile(loss = wasserstein_loss,optimizer = optim,loss_weights = [0.5])
  return disc

In [ ]:
def patch_gan(g_model, d_model, image_shape):
	"""Here we finally combine the generator and discriminator blocks into a single model"""
	d_model.trainable = False
	in_src = tf.keras.Input(shape=image_shape)
	gen_out = g_model(in_src)
	dis_out = d_model([in_src, gen_out])
	model = tf.keras.Model(in_src, [dis_out, gen_out])
	opt = tf.keras.optimizers.RMSprop(lr=0.2)
	model.compile(loss=[wasserstein_loss, 'mae'], optimizer=opt)
	return model

In [ ]:
#Defining the input image shape
img_dim = (128,128,3)

In [ ]:
#Creating an object of the discriminator block
disc_model = disc_sub(img_dim)

In [ ]:
#Summary of the architecture of the discriminator block 
disc_model.summary()

In [ ]:
#Creating an object of the generator block
generator_model = gen_sub(img_dim)

In [ ]:
#Creating an object of the patch gan block
patchgan_model = patch_gan(generator_model, disc_model, img_dim)

In [ ]:
#Summary of whole model architecture
patchgan_model.summary()

In [ ]:
x_train = np.asarray(data_array)
y_train = np.asarray(data2_array)

In [ ]:
#Printing the shape of the high resolution y_train images
print(y_train.shape)

In [ ]:
#Setting the patch size to 32
patch = 16

In [ ]:
#Setting the batch size and number of critics
batch_size = 32
n_critic = x_train.shape[0]//batch_size

In [ ]:
#Here we finally train the model and set the number of epochs 
n_critic = x_train.shape[0]//batch_size
clip_value = 0.01
epochs = 1000
for epoch in range(epochs):
  for i in range(n_critic):
    batch_image = x_train[i*batch_size:(i+1)*batch_size]

    batch_noise = y_train[i*batch_size:(i+1)*batch_size]
    batch_gen_image = generator_model.predict(batch_noise)

    d_loss_real = disc_model.train_on_batch([batch_image,batch_noise],np.ones((batch_size,patch,patch,1)))
    d_loss_fake = disc_model.train_on_batch([batch_image,batch_gen_image],np.zeros((batch_size,patch,patch,1)))
    d_loss = 0.5 * np.add(d_loss_real,d_loss_fake)
    for l in disc_model.layers:
      weights = l.get_weights()
      weights = [np.clip(w,-clip_value,clip_value) for w in weights]
      l.set_weights(weights)

    disc_model.trainable = False
    g_loss,_,_ = patchgan_model.train_on_batch(batch_noise,[np.ones((batch_size,patch,patch,1)),batch_image])
    disc_model.trainable = True

 
  print("epoch {} and loss {} ".format(epoch,g_loss))

In [ ]:
import math
def compute_psnr(img1, img2):
  """Here we calculate the psnr between the generated and the given image"""
  img1 = img1.astype(np.float64) / 255.
  img2 = img2.astype(np.float64) / 255.
  mse = np.mean((img1 - img2) ** 2)
  if mse == 0:
    return "Same Image"
  return 10 * math.log10(1. / mse)
def psnr_test(test_LR,test_HR):
  """Here we take the generated image and\\
   compute its PSNR with respect to the 
   given image and return the psnr value"""
   
  image_ = test_LR
  image_gen_ = generator_model.predict(image_)
  psnr_result = compute_psnr(np.asarray(image_gen_),test_HR)
  return psnr_result

In [ ]:
generator_model.save("generator_model1000.h5")

In [ ]:
image_gen_ = generator_model.predict(np.asarray(test_data_lr[256:1024]))

In [ ]:
peak_signal_noise_ratio(np.asarray(test_data_hr[256:1024]),image_gen_)